In [10]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [62]:
# Set the data path, and get the list of files to be preprocessed

trade_data_path = os.path.join(os.getcwd(),'daily_data','trade')
ohlcv_data_path = os.path.join(os.getcwd(),'daily_data','ohlcv')
preprocessed_data_path = os.path.join(os.getcwd(),'daily_data','preprocessed')

trade_data_files = sorted(os.listdir(trade_data_path))
ohlcv_data_files = sorted(os.listdir(ohlcv_data_path))

In [ ]:
# Original csv files downloaded from https://www.binance.com/en/landing/data do not include column names
# Column names and descriptions can be found in https://www.binance.com/en/support/faq/how-to-download-historical-market-data-on-binance-5810ae42176b4770b880ce1f14932262
# Manually set the column names based on it

trade_column_list = ['ID', 'price', 'qty', 'quote_qty', 'time', 'is_buyer_maker','is_best_match']
ohlcv_column_list = ['open_time','open','high','low','close','volume','close_time','quote_volume','num_trade','taker_buy_base','taker_buy_quote','ignore']

In [63]:
for i in tqdm(range(len(trade_data_files))):
    # Load the raw trade data
    trade = pd.read_csv(os.path.join(trade_data_path,trade_data_files[i]), names = trade_column_list)
    ohlcv = pd.read_csv(os.path.join(ohlcv_data_path,ohlcv_data_files[i]), names = ohlcv_column_list)
    trade_date = trade_data_files[i][-14:-4]
    ohlcv_date = ohlcv_data_files[i][-14:-4]
    assert trade_date == ohlcv_date                                                     # check if the dates of two csv files are identical
    
    trade['is_price_up'] = trade['price'].gt(trade['price'].shift(1)).astype(float)     # 1 if price increased from the last tick data, otherwise 0
    trade['is_price_down'] = trade['price'].lt(trade['price'].shift(1)).astype(float)   # 1 if price decreased from the last tick data, otherwise 0
    trade['price_up_down'] = trade['is_price_up'] - trade['is_price_down']              # 1 if price increased, -1 if price decreased, 0 if price remained same
    
    # Define "meaningful data" to be the tick data such that
    # either price increased and buyer is the maker,
    # or price decreased and seller is the maker
    trade['is_meaningful'] = trade['price_up_down'] * (2*trade['is_buyer_maker']-1)     # 1 only if the tick data is meaningful
    trade = trade[trade['is_meaningful']==1].reset_index().drop('index',axis=1)         # Leave only meaningful data
    
    ohlcv['ud'] = ohlcv['close'] - ohlcv['open']                                        # difference between opening and close price for each minute
    ohlcv['ma'] = (ohlcv['close'].rolling(5).mean() - ohlcv['close'].rolling(20).mean()).shift(1)  # Previous difference between 5min moving average and 20min moving average
    
    # Match each timestamp of the trade data to the corresponding index of 1-minute interval within the day
    trade['time'] = (trade['time'] - ohlcv['open_time'].loc[0]) // 60000
    
    # Calculate the ratio of upward movement(increase in price from the last tick data) within the meaningful data of each minute interval
    # and add this ratio to the ohlcv.csv as the "behav" column
    behav_df = trade.groupby('time')['price_up_down'].value_counts().unstack()
    behav_df['sum'] = (behav_df.sum(axis=1)).fillna(-1)
    ohlcv['behav'] = behav_df[1] / behav_df['sum']
    
    ohlcv = ohlcv.loc[20:]                                                              # Drop first 20 minute data for each day 
    ohlcv = ohlcv[['ud','ma','volume','behav']]                                         # Leave only the columns that will be used in the modeling
    ohlcv.loc[:,'volume'] = ohlcv['volume'].apply(lambda x: np.log(x))                  # Apply log to the volume data, considering the large range of the data
    ohlcv = ohlcv.astype('float32')
    
    ohlcv.to_csv(os.path.join(preprocessed_data_path,ohlcv_date+'.csv'), index=False, header=False)

100%|██████████| 31/31 [00:27<00:00,  1.11it/s]


In [64]:
# Merge the daily preprocessed csv files into a single csv file

preprocessed_data_files = sorted(os.listdir(preprocessed_data_path))
preprocessed_column_list = ['ud','ma','volume','behav']

data = pd.read_csv(os.path.join(preprocessed_data_path,preprocessed_data_files[0]),names=preprocessed_column_list)
for file in tqdm(preprocessed_data_files[1:]):
    df = pd.read_csv(os.path.join(preprocessed_data_path,file),names=preprocessed_column_list)
    data = pd.concat([data,df],axis=0)

data = data.reset_index().drop('index',axis=1)

100%|██████████| 30/30 [00:00<00:00, 556.32it/s]


In [65]:
# Replace the values of 1% anomaly data to the 1% boundary values

data['ud'] = data['ud'].apply(lambda x: max(min(x,150),-150))
data['ma'] = data['ma'].apply(lambda x: max(min(x,300),-300))
data['volume'] = data['volume'].apply(lambda x: max(min(x,5.5),1.2))

In [66]:
data.to_csv('input_data_2021-12.csv',index=False,header=False)